In [29]:
#Installing Dependencies

import numpy as np
import pandas as pd
import os
import pickle
import cv2

from sklearn.model_selection import train_test_split

from keras.utils import load_img
from keras.models import Sequential
from keras.layers.convolutional import Convolution2D
from keras.layers.pooling import MaxPooling2D
from keras.layers.core import Flatten, Dense, Activation, Dropout

In [4]:
#Pre-Processing

X_train = []
y_train = []

input_size = (96, 96)

In [5]:
folderpath = '/Users/shubhvashishth/Downloads/TrainingSet/Undistorted'

In [13]:
for filename in os.listdir(folderpath):
    imagepath = folderpath + '/' + filename
    img = load_img(imagepath, target_size = input_size)
    X_train.append((1/255)*np.asarray(img))
    # compute the Laplacian of the image and then return the focus
    img2 = cv2.imread(imagepath)
    gray = cv2.cvtColor(img2, cv2.COLOR_BGR2RGB)
    blur_map = cv2.Laplacian(gray, cv2.CV_64F)
    score = np.var(blur_map)
    y_train.append(score)
print("Trainset: Undistorted loaded...")

Trainset: Undistorted loaded...


In [17]:
folderpath = '/Users/shubhvashishth/Downloads/TrainingSet/Artificially-Blurred'

In [18]:
for filename in os.listdir(folderpath):
    imagepath = folderpath + '/' + filename
    img = load_img(imagepath, target_size = input_size)
    X_train.append((1/255)*np.asarray(img))
    # compute the Laplacian of the image and then return the focus
    img2 = cv2.imread(imagepath)
    gray = cv2.cvtColor(img2, cv2.COLOR_BGR2RGB)
    blur_map = cv2.Laplacian(gray, cv2.CV_64F)
    score = np.var(blur_map)
    y_train.append(score)
print("Trainset: Artificially Blurred loaded...")

Trainset: Artificially Blurred loaded...


In [21]:
folderpath = '/Users/shubhvashishth/Downloads/TrainingSet/Naturally-Blurred'

In [22]:
for filename in os.listdir(folderpath):
    imagepath = folderpath + '/' + filename
    img = load_img(imagepath, target_size = input_size)
    X_train.append((1/255)*np.asarray(img))
    # compute the Laplacian of the image and then return the focus
    img2 = cv2.imread(imagepath)
    gray = cv2.cvtColor(img2, cv2.COLOR_BGR2RGB)
    blur_map = cv2.Laplacian(gray, cv2.CV_64F)
    score = np.var(blur_map)
    y_train.append(score)
print("Trainset: Naturally Blurred loaded...")

Trainset: Naturally Blurred loaded...


In [27]:
#Converting the training data to a pickel file

with open('X_train.pkl', 'wb') as picklefile:
    pickle.dump(X_train, picklefile)

In [124]:
with open('y_train.pkl', 'wb') as picklefile:
    pickle.dump(y_train, picklefile)

In [108]:
# Defining the CNN architecture

model = Sequential()

# Layer 1
model.add(Convolution2D(32, (5, 5), input_shape=(input_size[0], input_size[1], 3)))
model.add(Activation("relu"))
model.add(MaxPooling2D(pool_size=(2, 2)))

# Layer 2
model.add(Convolution2D(64, (5, 5)))
model.add(Activation("relu"))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())

# Layer 3
model.add(Dense(1024))
model.add(Activation("relu"))
model.add(Dropout(0.5))

# Layer 4

model.add(Dense(512))
model.add(Activation("relu"))
model.add(Dropout(0.5))

model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='linear'))

# Compile the model
model.compile(loss='mean_squared_error', optimizer='adam',metrics=['mae'])



In [109]:
model.summary()

Model: "sequential_9"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_15 (Conv2D)          (None, 92, 92, 32)        2432      
                                                                 
 activation_29 (Activation)  (None, 92, 92, 32)        0         
                                                                 
 max_pooling2d_15 (MaxPoolin  (None, 46, 46, 32)       0         
 g2D)                                                            
                                                                 
 conv2d_16 (Conv2D)          (None, 42, 42, 64)        51264     
                                                                 
 activation_30 (Activation)  (None, 42, 42, 64)        0         
                                                                 
 max_pooling2d_16 (MaxPoolin  (None, 21, 21, 64)       0         
 g2D)                                                 

In [111]:
def normalize_list(lst):
    min_val = min(lst)
    max_val = max(lst)
    range_val = max_val - min_val
    normalized_lst = [(x - min_val) / range_val for x in lst]
    return normalized_lst

In [112]:
normal_y = normalize_list(y_train)

In [113]:
train_x = np.asarray(X_train)
train_y = np.asarray(normal_y)

In [114]:
x_training, x_testing, y_training, y_testing = train_test_split(train_x, train_y, test_size=0.2, random_state=42)

In [115]:
model.fit(x_training, y_training, batch_size=32, epochs=50, verbose=1)
print("Model training complete...")

Epoch 1/50


2023-04-25 22:53:27.980940: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


25/25 [==============================] - 3s 97ms/step - loss: 0.1370 - mae: 0.1541
Epoch 2/50
25/25 [==============================] - 2s 95ms/step - loss: 0.0074 - mae: 0.0502
Epoch 3/50
25/25 [==============================] - 2s 97ms/step - loss: 0.0080 - mae: 0.0507
Epoch 4/50
25/25 [==============================] - 2s 96ms/step - loss: 0.0082 - mae: 0.0566
Epoch 5/50
25/25 [==============================] - 2s 89ms/step - loss: 0.0081 - mae: 0.0563
Epoch 6/50
25/25 [==============================] - 2s 89ms/step - loss: 0.0080 - mae: 0.0547
Epoch 7/50
25/25 [==============================] - 2s 86ms/step - loss: 0.0080 - mae: 0.0569
Epoch 8/50
25/25 [==============================] - 2s 88ms/step - loss: 0.0083 - mae: 0.0575
Epoch 9/50
25/25 [==============================] - 2s 87ms/step - loss: 0.0080 - mae: 0.0555
Epoch 10/50
25/25 [==============================] - 2s 87ms/step - loss: 0.0075 - mae: 0.0531
Epoch 11/50
25/25 [==============================] - 2s 91ms/step - lo

In [117]:
test_loss, test_mae = model.evaluate(x_testing, y_testing)

7/7 [==============================] - 0s 22ms/step - loss: 0.0083 - mae: 0.0438


In [118]:
test_loss,test_mae

(0.00825471431016922, 0.04382963478565216)

In [119]:
# Calculate the accuracy score
accuracy_score = 1.0 / (1.0 + test_mae)
print("Accuracy Score:", accuracy_score)

Accuracy Score: 0.9580107391809656


In [120]:
model.save('model.h5')